<h1> Sydney Metro Data Science </h1>


Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          82 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge


geopy-1.17.0         | 49 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  envi

## 1. Download and Explore Dataset

In [101]:
import pandas as pd

 
df = pd.read_csv('postcodes.csv')
df.head()

    
    

,id,postcode,suburb,state,latitude,longitude
0,398,1001,Sydney,NSW,-33.79,151.27
1,399,1002,Sydney,NSW,-33.79,151.27
2,400,1003,Sydney,NSW,-33.79,151.27
3,401,1004,Sydney,NSW,-33.79,151.27
4,402,1005,Sydney,NSW,-33.79,151.27


Define a function to get GPS for a suburb

#### Data filtering

In [72]:
df=df[['postcode','suburb','longitude','latitude']]
df=df.drop_duplicates(subset=['suburb'], keep='first')
df = df[df.longitude != 0.00]
df = df[df.postcode >= 2000]
#df.head(10)
df.shape


(4527, 4)

We'd like to select a suburb called Beecroft as the central point for the data analysis

Get GEO location for the suburb

In [73]:
address = 'Beecroft, NSW'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Beecroft are {}, {}.'.format(latitude, longitude))


/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Beecroft are -33.7497631, 151.0657475.


In [74]:
df.head(10)

,postcode,suburb,longitude,latitude
318,2000,Barangaroo,151.21,-33.86
319,2000,Dawes Point,151.21,-33.86
321,2000,Millers Point,151.21,-33.86
322,2000,Parliament House,151.21,-33.86
325,2000,The Rocks,151.21,-33.86
327,2002,World Square,146.41,-35.97
330,2006,The University Of Sydney,151.19,-33.89
332,2007,Ultimo,151.20,-33.88
333,2008,Chippendale,151.20,-33.89
334,2008,Darlington,151.20,-33.89


Manually pick the suburbs nearby

In [75]:
neighborhoods = df[df['postcode'].isin([2117,2119,2118,2120,2121,2122,2125,2126,2076])].reset_index(drop=True)


neighborhoods.shape


(20, 4)

#### Create a map of Beecroft with neighborhoods superimposed on top.

In [76]:

map_beecroft = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, suburb in zip(neighborhoods['latitude'], neighborhoods['longitude'], neighborhoods['suburb']):
    label = '{}'.format(suburb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_beecroft)  
    
map_beecroft

#### Define Foursquare Credentials and Version

In [77]:
CLIENT_ID = 'VDFTNYUMJI3FIAI2E1GDVXWGMBJZYB30DCLYIJA5IIYQXX3F' # your Foursquare ID
CLIENT_SECRET = 'ZQSBPY4I2RWWPXV1ZHWCHV4YQORZPEOSWGFNDV33GWE2T4CO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VDFTNYUMJI3FIAI2E1GDVXWGMBJZYB30DCLYIJA5IIYQXX3F
CLIENT_SECRET:ZQSBPY4I2RWWPXV1ZHWCHV4YQORZPEOSWGFNDV33GWE2T4CO


#### Let's explore the neighborhood in our dataframe.

Get the neighborhood's latitude and longitude values.

In [78]:

neighborhood_latitude = df.loc[df['suburb'] == 'Beecroft', 'latitude'].iloc[0] # neighborhood latitude value
neighborhood_longitude = df.loc[df['suburb'] == 'Beecroft', 'longitude'].iloc[0] # neighborhood longitude value



neighborhood_name = df.loc[df['suburb'] == 'Beecroft', 'suburb'].iloc[0] # neighborhood name


print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Beecroft are -33.75, 151.06.


#### Now, let's get the top 100 venues that are in Beecroft within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [79]:
# type your answer here
LIMIT = 100
radius = 500


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=VDFTNYUMJI3FIAI2E1GDVXWGMBJZYB30DCLYIJA5IIYQXX3F&client_secret=ZQSBPY4I2RWWPXV1ZHWCHV4YQORZPEOSWGFNDV33GWE2T4CO&v=20180605&ll=-33.75,151.06&radius=500&limit=100'

Send the GET request and examine the resutls

In [80]:
results = requests.get(url).json()
#results

let's borrow the get_category_type function from the Foursquare lab.

In [81]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [82]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Woolworths,Supermarket,-33.749232,151.065080
1,Yummy Kitchens,Chinese Restaurant,-33.750190,151.064923
2,Beecroft Place,Shopping Mall,-33.749210,151.064855
3,Cafe Bela,Café,-33.749362,151.065152
4,Beehive Cafe,Café,-33.750334,151.065210
5,Sevardi Cucina Italiana,Italian Restaurant,-33.749405,151.065345
6,Beecroft Gourmet Chicken & Salads,Fast Food Restaurant,-33.749413,151.065347
7,Franklins,Convenience Store,-33.749312,151.065334


And how many venues were returned by Foursquare?

In [83]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Beecroft

#### Let's create a function to repeat the same process to all the neighborhoods in Beecroft

In [84]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *beecroft_venues*.

The neighbor suburbs are 2117,2119,2118,2120,2121,2122,2125,2126,2076

In [85]:
neighborhoods = df[df['postcode'].isin([2117,2119,2118,2120,2121,2122,2125,2126,2076])].reset_index(drop=True)

neighborhoods

beecroft_venues = getNearbyVenues(names=neighborhoods['suburb'],
                                   latitudes=neighborhoods['latitude'],
                                   longitudes=neighborhoods['longitude']
                                  )


Normanhurst
North Wahroonga
Wahroonga
Dundas
Dundas Valley
Oatlands
Telopea
Carlingford
Carlingford Court
Carlingford North
Kingsdene
Beecroft
Cheltenham
Thornleigh
Westleigh
North Epping
Eastwood
Marsfield
West Pennant Hills
Cherrybrook


In [86]:

beecroft_venues.shape

(764, 7)

In [87]:
beecroft_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Beecroft,38,38,38,38,38,38
Carlingford,29,29,29,29,29,29
Carlingford Court,29,29,29,29,29,29
Carlingford North,29,29,29,29,29,29
Cheltenham,38,38,38,38,38,38
Cherrybrook,16,16,16,16,16,16
Dundas,18,18,18,18,18,18
Dundas Valley,18,18,18,18,18,18
Eastwood,74,74,74,74,74,74


In [88]:
print('There are {} uniques categories.'.format(len(beecroft_venues['Venue Category'].unique())))

There are 73 uniques categories.


## 3. Analyze Each Neighborhood


In [89]:
# one hot encoding
beecroft_onehot = pd.get_dummies(beecroft_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
beecroft_onehot['Neighborhood'] = beecroft_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [beecroft_onehot.columns[-1]] + list(beecroft_onehot.columns[:-1])
beecroft_onehot = beecroft_onehot[fixed_columns]

beecroft_onehot.shape

(764, 74)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [90]:
beecroft_grouped = beecroft_onehot.groupby('Neighborhood').mean().reset_index()
beecroft_grouped.shape

(20, 74)

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [92]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = beecroft_grouped['Neighborhood']

for ind in np.arange(beecroft_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(beecroft_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted

## 4. Cluster Neighborhoods


Run *k*-means to cluster the neighborhood into 3 clusters.

In [102]:
# set number of clusters
kclusters = 3

beecroft_grouped_clustering = beecroft_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(beecroft_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:12] 

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1], dtype=int32)

In [94]:
beecroft_grouped

,Neighborhood,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Bakery,Big Box Store,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Football Stadium,Fried Chicken Joint,Frozen Yogurt Shop,Garden,General Entertainment,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Library,Liquor Store,Malay Restaurant,Middle Eastern Restaurant,Movie Theater,Noodle House,Park,Persian Restaurant,Pizza Place,Platform,Pool,Post Office,Pub,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Soccer Field,Social Club,Spa,Sports Bar,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio,Zoo
0,Beecroft,0.026316,0.026316,0.000000,0.000000,0.000000,0.000000,0.026316,0.052632,0.000000,0.000000,0.000000,0.026316,0.026316,0.184211,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.026316,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.026316,0.026316,0.000000,0.105263,0.026316,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.026316,0.000000,0.052632,0.026316,0.000000,0.052632,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,0.026316
1,Carlingford,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.034483,0.034483,0.000000,0.034483,0.000000,0.000000,0.103448,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103448,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.034483,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.034483,0.034483,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.034483,0.034483,0.000000,0.000000,0.000000,0.068966,0.000000,0.000000,0.000000,0.000000,0.000000,0.068966,0.000000,0.034483,0.000000,0.000000,0.000000
2,Carlingford Court,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.034483,0.034483,0.000000,0.034483,0.000000,0.000000,0.103448,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103448,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.034483,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.034483,0.034483,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.034483,0.034483,0.000000,0.000000,0.000000,0.068966,0.000000,0.000000,0.000000,0.000000,0.000000,0.068966,0.000000,0.034483,0.000000,0.000000,0.000000
3,Carlingford North,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.034483,0.034483,0.000000,0.034483,0.000000,0.000000,0.103448,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103448,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.034483,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.034483,0.034483,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.034483,0.034483,0.000000,0.000000,0.000000,0.068966,0.000000,0.000000,0.000000,0.000000,0.000000,0.068966,0.000000,0.034483,0.000000,0.000000,0.000000
4,Cheltenham,0.026316,0.026316,0.000000,0.000000,0.000000,0.000000,0.026316,0.052632,0.000000,0.000000,0.000000,0.026316,0.026316,0.184211,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.026316,0.000000,0.000000,0.000000,0.000000,0.00000

In [95]:

neighborhoods.columns=['PostCode','Neighborhood','Longitude','Latitude']
neighborhoods.shape
      

(20, 4)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [96]:
beecroft_merged = neighborhoods

# add clustering labels
beecroft_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
beecroft_merged = beecroft_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

beecroft_merged

,PostCode,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2076,Normanhurst,151.10,-33.72,1,Café,Gym,Indian Restaurant,Asian Restaurant,Park,Chinese Restaurant,Bakery,Grocery Store,Electronics Store,Japanese Restaurant
1,2076,North Wahroonga,151.10,-33.72,0,Café,Gym,Indian Restaurant,Asian Restaurant,Park,Chinese Restaurant,Bakery,Grocery Store,Electronics Store,Japanese Restaurant
2,2076,Wahroonga,151.10,-33.72,0,Café,Gym,Indian Restaurant,Asian Restaurant,Park,Chinese Restaurant,Bakery,Grocery Store,Electronics Store,Japanese Restaurant
3,2117,Dundas,151.04,-33.80,0,Pizza Place,Grocery Store,Café,Supermarket,Pub,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Soccer Field,Bus Station
4,2117,Dundas Valley,151.04,-33.80,1,Pizza Place,Grocery Store,Café,Supermarket,Pub,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Soccer Field,Bus Station
5,2117,Oatlands,151.04,-33.80,0,Pizza Place,Grocery Store,Café,Supermarket,Pub,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Soccer Field,Bus Station
6,2117,Telopea,151.04,-33.80,0,Pizza Place,Grocery Store,Café,Supermarket,Pub,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Soccer Field,Bus Station
7,2118,Carlingford,151.05,-33.78,0,Fast Food Restaurant,Café,Train Station,Supermarket,Italian Restaurant,Shopping Mall,Gym,Gym / Fitness Center,Food Court,Korean Restaurant
8,2118,Carlingford Court,151.05,-33.78,1,Fast Food Restaurant,Café,Train Station,Supermarket,Italian Restaurant,Shopping Mall,Gym,Gym / Fitness Center,Food Court,Korean Restaurant
9,2118,Carlingford North,151.05,-33.78,0,Fast Food Restaurant,Café,Train Station,Supermarket,Italian Restaurant,Shopping Mall,Gym,Gym / Fitness Center,Food Court,Korean Restaurant


Finally, let's visualize the resulting clusters

In [97]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(beecroft_merged['Latitude'], beecroft_merged['Longitude'], beecroft_merged['Neighborhood'], beecroft_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [98]:
beecroft_merged.loc[beecroft_merged['Cluster Labels'] == 0, beecroft_merged.columns[[1] + list(range(5, beecroft_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North Wahroonga,Café,Gym,Indian Restaurant,Asian Restaurant,Park,Chinese Restaurant,Bakery,Grocery Store,Electronics Store,Japanese Restaurant
2,Wahroonga,Café,Gym,Indian Restaurant,Asian Restaurant,Park,Chinese Restaurant,Bakery,Grocery Store,Electronics Store,Japanese Restaurant
3,Dundas,Pizza Place,Grocery Store,Café,Supermarket,Pub,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Soccer Field,Bus Station
5,Oatlands,Pizza Place,Grocery Store,Café,Supermarket,Pub,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Soccer Field,Bus Station
6,Telopea,Pizza Place,Grocery Store,Café,Supermarket,Pub,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Soccer Field,Bus Station
7,Carlingford,Fast Food Restaurant,Café,Train Station,Supermarket,Italian Restaurant,Shopping Mall,Gym,Gym / Fitness Center,Food Court,Korean Restaurant
9,Carlingford North,Fast Food Restaurant,Café,Train Station,Supermarket,Italian Restaurant,Shopping Mall,Gym,Gym / Fitness Center,Food Court,Korean Restaurant
14,Westleigh,Café,Pizza Place,Thai Restaurant,Liquor Store,Chinese Restaurant,Platform,Gym / Fitness Center,Fast Food Restaurant,Shopping Mall,Bakery
15,North Epping,Thai Restaurant,Indian Restaurant,Pizza Place,Italian Restaurant,Social Club,Fried Chicken Joint,Football Stadium,Korean Restaurant,Malay Restaurant,Dessert Shop


#### Cluster 2

In [99]:
beecroft_merged.loc[beecroft_merged['Cluster Labels'] == 1, beecroft_merged.columns[[1] + list(range(5, beecroft_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Normanhurst,Café,Gym,Indian Restaurant,Asian Restaurant,Park,Chinese Restaurant,Bakery,Grocery Store,Electronics Store,Japanese Restaurant
4,Dundas Valley,Pizza Place,Grocery Store,Café,Supermarket,Pub,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Soccer Field,Bus Station
8,Carlingford Court,Fast Food Restaurant,Café,Train Station,Supermarket,Italian Restaurant,Shopping Mall,Gym,Gym / Fitness Center,Food Court,Korean Restaurant
10,Kingsdene,Fast Food Restaurant,Café,Train Station,Supermarket,Italian Restaurant,Shopping Mall,Gym,Gym / Fitness Center,Food Court,Korean Restaurant
11,Beecroft,Café,Pizza Place,Bakery,Thai Restaurant,Supermarket,Zoo,Fast Food Restaurant,Park,Noodle House,Liquor Store
12,Cheltenham,Café,Pizza Place,Bakery,Thai Restaurant,Supermarket,Zoo,Fast Food Restaurant,Park,Noodle House,Liquor Store
13,Thornleigh,Café,Pizza Place,Thai Restaurant,Liquor Store,Chinese Restaurant,Platform,Gym / Fitness Center,Fast Food Restaurant,Shopping Mall,Bakery
16,Eastwood,Chinese Restaurant,Japanese Restaurant,Supermarket,BBQ Joint,Vietnamese Restaurant,Korean Restaurant,Café,Thai Restaurant,Indian Restaurant,Park
17,Marsfield,Chinese Restaurant,Japanese Restaurant,Supermarket,BBQ Joint,Vietnamese Restaurant,Korean Restaurant,Café,Thai Restaurant,Indian Restaurant,Park
19,Cherrybrook,Shopping Mall,Liquor Store,Italian Restaurant,Grocery Store,Park,Noodle House,Sandwich Place,Café,Supermarket,Electronics Store


#### Cluster 3

In [100]:
beecroft_merged.loc[beecroft_merged['Cluster Labels'] == 2, beecroft_merged.columns[[1] + list(range(5, beecroft_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,West Pennant Hills,Pizza Place,Park,Zoo,Bakery,Bus Station,Spa,Sports Bar,Liquor Store,Supermarket,Trail
